# Development

In [47]:
#import 
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd 
import bs4 as bs
import nltk
from nltk.tokenize import sent_tokenize # tokenizes sentences
import re
from nltk.stem import PorterStemmer
from nltk.tag import pos_tag
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

eng_stopwords = stopwords.words('english')

# Load more data

source data from https://crisisnlp.qcri.org/lrec2016/lrec2016.html

In [48]:
# eq - earthquake

eq_pakistan_2013 = pd.read_csv('data/2013_pakistan_eq.csv', skip_blank_lines=True, encoding = "ISO-8859-1")
eq_pakistan_2013['cat'] = 'eq_pakistan_2013'

eq_california_2014 = pd.read_csv('data/2014_california_eq.csv', skip_blank_lines=True, encoding = "ISO-8859-1")
eq_california_2014['cat'] = 'eq_california_2014'

eq_chile_2014 = pd.read_csv('data/2014_chile_eq_en.csv', skip_blank_lines=True, encoding = "ISO-8859-1")
eq_chile_2014['cat'] = 'eq_chile_2014'

ebola_virus_2014 = pd.read_csv('data/2014_ebola_virus.csv', skip_blank_lines=True, encoding = "ISO-8859-1")
ebola_virus_2014['cat'] = 'ebola_virus_2014'

hurricane_odile_2014 = pd.read_csv('data/2014_hurricane_odile.csv', skip_blank_lines=True, encoding = "ISO-8859-1")
hurricane_odile_2014['cat'] = 'hurricane_odile_2014'

flood_india_2014 = pd.read_csv('data/2014_india_floods.csv', skip_blank_lines=True, encoding = "ISO-8859-1")
flood_india_2014['cat'] = 'flood_india_2014'

middle_east_respiratory_2014 = pd.read_csv('data/2014_mers_cf_labels.csv', skip_blank_lines=True, encoding = "ISO-8859-1")
middle_east_respiratory_2014['cat'] = 'middle_east_respiratory_2014'

flood_pakistan_2014 = pd.read_csv('data/2014_pakistan_floods_cf_labels.csv', skip_blank_lines=True, encoding = "ISO-8859-1")
flood_pakistan_2014['cat'] = 'flood_pakistan_2014'

typhoon_philippines_2014 = pd.read_csv('data/2014_typhoon_hagupit_cf_labels.csv', skip_blank_lines=True, encoding = "ISO-8859-1")
typhoon_philippines_2014['cat']='typhoon_philippines_2014'

cyclone_pam_2015 = pd.read_csv('data/2015_cyclone_pam_cf_labels.csv', skip_blank_lines=True, encoding = "ISO-8859-1")
cyclone_pam_2015['cat'] = 'cyclone_pam_2015'

eq_nepal_2015 = pd.read_csv('data/2015_nepal_eq_cf_labels.csv', skip_blank_lines=True, encoding = "ISO-8859-1")
eq_nepal_2015['cat'] = 'eq_nepal_2015'

In [52]:
file = pd.concat([cyclone_pam_2015, eq_nepal_2015, typhoon_philippines_2014, flood_pakistan_2014,
                middle_east_respiratory_2014, flood_india_2014, hurricane_odile_2014, 
                ebola_virus_2014, eq_chile_2014, eq_california_2014, eq_pakistan_2013])

In [53]:
len(file)

23146

In [54]:
file.groupby(by='choose_one_category').count()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,choose_one_category:confidence,choose_one_category_gold,tweet_id,tweet_text,cat
choose_one_category,,,,,,,,,,
affected_people,866,866,866,866,860,866,6,866,866,866
caution_and_advice,1063,1063,1063,1063,1053,1063,10,1063,1063,1063
deaths_reports,94,94,94,94,93,94,1,93,93,94
disease_signs_or_symptoms,431,431,431,431,425,431,8,431,431,431
disease_transmission,425,425,425,425,420,425,6,425,425,425
displaced_people_and_evacuations,633,633,633,633,625,633,8,633,633,633
donation_needs_or_offers_or_volunteering_services,2610,2610,2610,2610,2591,2610,19,2610,2610,2610
infrastructure_and_utilities_damage,1851,1851,1851,1851,1835,1851,16,1851,1851,1851
injured_or_dead_people,2510,2510,2510,2510,2490,2510,19,2510,2510,2510


In [55]:
file.columns

Index(['_unit_id', '_golden', '_unit_state', '_trusted_judgments',
       '_last_judgment_at', 'choose_one_category',
       'choose_one_category:confidence', 'choose_one_category_gold',
       'tweet_id', 'tweet_text', 'cat'],
      dtype='object')

In [56]:
file = file.drop(['_unit_id', '_golden', '_trusted_judgments',
       '_last_judgment_at', 'choose_one_category:confidence', 'choose_one_category_gold',
       'tweet_id'], axis=1)

In [58]:
file = file.dropna()

In [59]:
file.groupby(['choose_one_category']).count()

,_unit_state,tweet_text,cat
choose_one_category,,,
affected_people,866,866,866
caution_and_advice,1063,1063,1063
deaths_reports,93,93,93
disease_signs_or_symptoms,431,431,431
disease_transmission,425,425,425
displaced_people_and_evacuations,633,633,633
donation_needs_or_offers_or_volunteering_services,2610,2610,2610
infrastructure_and_utilities_damage,1851,1851,1851
injured_or_dead_people,2510,2510,2510


In [61]:
#reduced = file[file.choose_one_category == 'injured_or_dead_people']

#try with all data points
reduced = file

### cleaning steps

In [62]:
reduced.iloc[4].tweet_text

'Aid agencies: Vanuatu conditions more challenging than Philippines typhoon http://t.co/G9WIl4qrHx'

In [63]:
#remove twitter specific 
#file.tweet_text = re.sub(r'http\S+', '', file.tweet_text)
#remove hyperlinks
reduced = reduced.replace(to_replace =r'&amp;', value = '', regex = True)
reduced = reduced.replace(to_replace =r'&gt;', value = '', regex = True)
reduced = reduced.replace(to_replace =r'http\S+', value = '', regex = True)
#remove usernames
reduced = reduced.replace(to_replace =r'@\S+', value = '', regex = True) 
#remove hashtags
#reduced = reduced.replace(to_replace =r'#[A-Za-z0-9]+', value = '', regex = True) 
# or just remove the hashtag, but leave the actual word
reduced = reduced.replace(to_replace ='#', value = '', regex = True) 
#remove retweet
reduced = reduced.replace(to_replace ='RT :', value = '', regex = True) 
reduced = reduced.replace(to_replace ='RT ', value = '', regex = True) 
#remove punctation
reduced = reduced.replace(to_replace ='[",:!?\\-]', value = ' ', regex = True)

In [64]:
#reduced = reduced.replace(to_replace ='^[[-+]?[0-9]*\.?[0-9]*]', value = ' ', regex = True)
#'^[\d+\.$]'
#'^[[-+]?[0-9]*\.?[0-9]*]'

In [65]:
reduced.iloc[4].tweet_text

'Aid agencies  Vanuatu conditions more challenging than Philippines typhoon '

In [66]:
#4. Tokenize into words (all lower case)
reduced.tweet_text = reduced.tweet_text.str.lower()
reduced.tweet_text = reduced.tweet_text.str.split() 

In [67]:
eng_stopwords = set(stopwords.words("english"))
reduced['tweet_text'] = reduced['tweet_text'].apply(lambda x: [item for item in x if item not in eng_stopwords])

In [68]:
#join the list items back to one string
reduced['tweet_text'] = reduced['tweet_text'].apply(lambda x: ' '.join(x))

## Classify dataset

In [69]:
file.columns

Index(['_unit_state', 'choose_one_category', 'tweet_text', 'cat'], dtype='object')

In [70]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics # for confusion matrix, accuracy score etc
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(\
    reduced['tweet_text'], reduced['choose_one_category'], random_state=0, test_size=.2)


# CountVectorizer can actucally handle a lot of the preprocessing for us
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000)

In [71]:
%%time
# Transform the text data to feature
# Only fit training data (to mimic real world)

vectorizer.fit(X_train)

Wall time: 781 ms


CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=5000, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [74]:
# Check that it worked, 
# now we have fitted a model that can transform features
# to sparse matrix representation

print(vectorizer.get_feature_names()[:1000])

['00', '000', '00pm', '01', '011', '02', '03', '04', '05', '06', '07', '08', '09', '10', '100', '1000', '1000s', '1003', '100s', '101', '102', '105', '109', '10km', '10pm', '11', '110', '11am', '11pm', '12', '120', '122', '125', '125mph', '13', '130', '132', '135', '13th', '14', '140', '142', '1490', '15', '150', '1500', '155', '15km', '16', '160', '1670', '17', '170', '1715gmt', '172', '175', '18', '180', '1800', '182', '18km', '19', '1900', '1988', '1989', '1b', '1km', '1m', '1st', '1ã', '20', '200', '2000', '20000', '2005', '2008', '2010', '2012', '2013', '2014', '2015', '20km', '20m', '21', '210', '22', '23', '230', '24', '240', '24km', '25', '250', '26', '260', '27', '28', '285', '29', '294', '2m', '2nd', '2â', '2ã', '30', '300', '31', '32', '3200', '324ff166', '327', '328', '33', '333', '33km', '34', '348', '35', '350', '355', '36', '37', '370', '38', '39', '3km', '3m', '3novices', '3rd', '3ã', '40', '400', '4000', '407', '40km', '41', '42', '43', '44', '440', '449', '45', '450',

In [75]:
train_bag = vectorizer.transform(X_train) #transform to a feature matrix
test_bag = vectorizer.transform(X_test)

In [76]:
print(train_bag.toarray().shape) # 20,000 reviews, 2,000 feartures. just as expected
print(test_bag.toarray().shape)

(18516, 5000)
(4629, 5000)


In [77]:
type(train_bag) # sparse matrix representation

print(train_bag)

  (0, 813)	1
  (0, 2131)	1
  (0, 2507)	1
  (0, 3028)	1
  (0, 3062)	1
  (0, 3137)	1
  (0, 3536)	1
  (0, 4670)	1
  (0, 4799)	1
  (1, 1296)	1
  (1, 1367)	1
  (1, 2458)	1
  (1, 3131)	1
  (1, 3247)	1
  (1, 4165)	1
  (2, 13)	1
  (2, 1218)	2
  (2, 1469)	1
  (2, 1470)	1
  (2, 2281)	1
  (2, 2819)	2
  (2, 3546)	1
  (2, 4014)	1
  (2, 4731)	1
  (3, 818)	1
  :	:
  (18513, 4731)	1
  (18514, 1058)	1
  (18514, 1518)	1
  (18514, 2067)	1
  (18514, 2790)	2
  (18514, 3051)	1
  (18514, 3175)	1
  (18514, 3506)	1
  (18514, 4206)	1
  (18514, 4601)	1
  (18514, 4667)	1
  (18514, 4731)	1
  (18515, 92)	1
  (18515, 500)	1
  (18515, 1584)	1
  (18515, 1908)	1
  (18515, 2218)	1
  (18515, 2280)	1
  (18515, 2458)	1
  (18515, 2850)	1
  (18515, 3279)	1
  (18515, 3347)	1
  (18515, 3660)	1
  (18515, 4477)	1
  (18515, 4504)	1


## Clasify with Random Forest model

* Fit a Random Forest model to our bagged data set in order to do the sentiment analysis on `review_clean_original` and print the **validation accuracy** by using `forest.predict(test_bag)` and then comparing the resulting sentiment predictions with the ones stored in `y_test`.

*This can take 2-3 mins to run*

In [78]:
reduced.columns

Index(['_unit_state', 'choose_one_category', 'tweet_text', 'cat'], dtype='object')

In [79]:
#list(reduced.choose_one_category.unique())

In [81]:
from sklearn.ensemble import RandomForestClassifier

## Initialize a Random Forest classifier with 50 trees
# hyperparameter n_estimators always set in instantiation

forest = RandomForestClassifier(n_estimators = 50) 

C:\Users\Florian\Anaconda3\envs\data-x\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [82]:
%%time
# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the target variable

forest = forest.fit(train_bag, y_train) # can take 60 seconds to run

Wall time: 57 s


In [84]:
# Make predictions

train_predictions = forest.predict(train_bag)
valid_predictions = forest.predict(test_bag)

## Accuracy

In [85]:
metrics.accuracy_score(y_train,train_predictions) # 100% training accuracy

0.9740224670555195

In [86]:
metrics.accuracy_score(y_test,valid_predictions) # 83% test

0.700151220565997

In [96]:
# Confusion matrix
# Is the number of False Positives and True negatives approx 50/50?
display(metrics.confusion_matrix(y_test,valid_predictions))

array([[ 126,    0,    0,   11,    3,    0,    0,    0,    0,    0,    1,
          13,    1,    1,   10],
       [   0,   64,    0,    0,    0,    0,    4,    4,    1,    0,   12,
         116,    0,   13,    0],
       [   2,    0,    5,    2,    0,    0,    0,    0,    1,    0,    0,
           1,    0,    0,    1],
       [  12,    0,    0,   61,    2,    0,    0,    0,    0,    0,    0,
           9,    0,    0,    2],
       [   3,    0,    0,    1,   65,    0,    0,    0,    0,    0,    0,
          13,    2,    0,    1],
       [   0,    4,    0,    0,    0,   56,   11,    9,    4,    2,    4,
          31,    0,    0,    0],
       [   0,    3,    0,    0,    0,    3,  408,    8,    0,    1,   19,
          84,    0,    6,    0],
       [   0,    3,    0,    0,    0,    2,   10,  245,    8,    0,   11,
          97,    0,    0,    0],
       [   0,    1,    0,    0,    0,    0,    7,    3,  423,    2,    8,
          35,    0,    3,    0],
       [   0,    0,    0,    0,    0,

In [91]:
forest.classes_

array(['affected_people', 'caution_and_advice', 'deaths_reports',
       'disease_signs_or_symptoms', 'disease_transmission',
       'displaced_people_and_evacuations',
       'donation_needs_or_offers_or_volunteering_services',
       'infrastructure_and_utilities_damage', 'injured_or_dead_people',
       'missing_trapped_or_found_people', 'not_related_or_irrelevant',
       'other_useful_information', 'prevention',
       'sympathy_and_emotional_support', 'treatment'], dtype=object)

In [94]:
# What are the characteristics of right / wrong classifications
# Good practice when doing analysis

df_test = pd.DataFrame(X_test)
df_test[(y_test.values=='infrastructure_and_utilities_damage') & (valid_predictions=='infrastructure_and_utilities_damage')]

,tweet_text
1669,path hurricane odile's destruction seen instag...
1991,(losdelsonido) hurricane odile slams mexico's ...
532,news california even buildings seismic retrofi...
1500,even buildings seismic retrofits damaged napa ...
1748,100 000+ people likely homeless 90% houses dam...
1743,awful damage hurricane odile baja california
974,tragic sight los cabos international airport h...
845,napaquake damage. broken chimneys problems ktv...
653,us quake damage ã¢â¬å rock on. california th...
508,cabosanlucas damage pics dallas residents jean...


## Feature importances

In [289]:
importances = forest.feature_importances_
# returns relative importance of all features.
# they are in the order of the columns
print(importances)

[3.23302134e-06 3.80829646e-04 3.74849619e-05 ... 1.34828742e-04
 2.61403472e-05 4.77084384e-05]


In [290]:
# sort importance scores
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")
top_10 = indices[:10]

# Get top ten features
print([vectorizer.get_feature_names()[ind] for ind in top_10])

Feature ranking:
['damage', 'injured', 'dozens', 'photo', 'earthquake', 'help', 'california', 'napa', 'injuries', 'hurt']


# WIP: Method - to be adjusted

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# put everything together in a function

def predict_sentiment(cleaned_reviews, y=train["sentiment"]):

    print("Creating the bag of words model!\n")
    # CountVectorizer" is scikit-learn's bag of words tool, here we show more keywords 
    vectorizer = CountVectorizer(analyzer = "word",   \
                                 tokenizer = None,    \
                                 preprocessor = None, \
                                 stop_words = None,   \
                                 max_features = 2000) 
    
    X_train, X_test, y_train, y_test = train_test_split(\
    cleaned_reviews, y, random_state=0, test_size=.2)

    # Then we use fit_transform() to fit the model / learn the vocabulary,
    # then transform the data into feature vectors.
    # The input should be a list of strings. .toarraty() converts to a numpy array
    
    train_bag = vectorizer.fit_transform(X_train).toarray()
    test_bag = vectorizer.transform(X_test).toarray()

    # You can extract the vocabulary created by CountVectorizer
    # by running print(vectorizer.get_feature_names())


    print("Training the random forest classifier!\n")
    # Initialize a Random Forest classifier with 75 trees
    forest = RandomForestClassifier(n_estimators = 50) 

    # Fit the forest to the training set, using the bag of words as 
    # features and the sentiment labels as the target variable
    forest = forest.fit(train_bag, y_train)


    train_predictions = forest.predict(train_bag)
    test_predictions = forest.predict(test_bag)
    
    train_acc = metrics.accuracy_score(y_train, train_predictions)
    valid_acc = metrics.accuracy_score(y_test, test_predictions)
    print("The training accuracy is: ", train_acc, "\n", "The validation accuracy is: ", valid_acc)
    
    return(forest,vectorizer)

# Ideabox below

## Compare Original cleaned to lemmatized and stemmed data set

Now carry out the same analysis as above but on the `review_clean_ps` and `review_clean_wnl`. 

What data preprocessing strategy worked the best? Why do you think that is? (Feel free to change the number of features extracted in the bag of words model and the number of trees in the random forest model (i.e. the hyperparameters in our model), to see how it effects your accuracy. Is the accuracy better or worse?

In [ ]:
%%time

print('Original Reviews')
forest1,vec1 = predict_sentiment(review_clean_original)
print('Porter Stemmer')
forest2,vec2 = predict_sentiment(review_clean_ps)
print('Lemmatizing')
forest3,vec3 = predict_sentiment(review_clean_wnl)


# It  seems like Porter Stemmer and Lemmatizing does not effect the results as much as we thought
# This is just what Sebastian Raschka points out in his book Python Machine Learning:

'''
The Porter stemming algorithm is probably the oldest and simplest
stemming algorithm. Other popular stemming algorithms include the
newer Snowball stemmer (Porter2 or "English" stemmer) or the Lancaster
stemmer (Paice-Husk stemmer), which is faster but also more aggressive
than the Porter stemmer. Those alternative stemming algorithms are also
available through the NLTK package (http://www.nltk.org/api/
nltk.stem.html).

While stemming can create non-real words, such as thu, (from thus) as
shown in the previous example, a technique called lemmatization aims to
obtain the canonical (grammatically correct) forms of individual words—
the so-called lemmas. However, lemmatization is computationally more
diffcult and expensive compared to stemming and, in practice, it has
been observed that stemming and lemmatization have little impact on the
performance of text classifcation (Michal Toman, Roman Tesar, and Karel
Jezek. Infuence of word normalization on text classifcation. Proceedings of
InSciT, pages 354–358, 2006).
'''

In [ ]:
vec1.get_feature_names()

In [ ]:
for vectorizer in [vec1, vec2, vec3]:
    print('TOP TEN IMPORTANT FEATURES:')
    importances = forest.feature_importances_
    indices = np.argsort(importances)[::-1]
    top_10 = indices[:10]
    print([vectorizer.get_feature_names()[ind] for ind in top_10])

## Named entity recognition/ disambiguiation
- find out name of school, city, street etc.

## Word embeddings

## Sentiment analysis
- - sentiment analysis 
    - check paper at https://www.analyticsvidhya.com/blog/2017/01/sentiment-analysis-of-twitter-posts-on-chennai-floods-using-python/, where sentiment analysis was performed on Chennai flood dataset

In [ ]:
#output: counting expressions (like Sandy Hook School or shooting)

## Lemmatization

In [38]:
def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return 'n'

In [39]:
wnl = WordNetLemmatizer()

wnl_stems = []
for pair in token_tag:
    res = wnl.lemmatize(pair[0],pos=get_wordnet_pos(pair[1]))
    wnl_stems.append(res)

print(' '.join(wnl_stems))

hi sandy hook school i think there be somebody shoot in here in sandy hook school because somebody get a gun i catch a glimpse of someone theyre run down the hallway they be still run theyre still shoot sandy hook school please


## Stopwords

In [42]:
tsc_wo_stopwords = [w for w in tsc_words if not w in stopwords.words("english")]
removed_stopwords = [w for w in tsc_words if w in stopwords.words("english")]

print('REVIEW WITHOUT STOPWORDS:')
print(' '.join(tsc_wo_stopwords))
print()
print('Stop words removed', removed_stopwords)
print()
print('NUMBER OF STOPWORDS REMOVED:',len(removed_stopwords))

REVIEW WITHOUT STOPWORDS:
hi sandy hook school think somebody shooting sandy hook school somebodys got gun caught glimpse someone theyre running hallway still running theyre still shooting sandy hook school please

Stop words removed ['i', 'there', 'is', 'in', 'here', 'in', 'because', 'a', 'i', 'a', 'of', 'down', 'the', 'they', 'are']

NUMBER OF STOPWORDS REMOVED: 15


# ---------------------------------------------------------

### here follows a summary of what we extracted from the text (summary, keywords etc.) and how this influences the priority

### processing: spot what is an emergency situation

### recommend steps what and how to do it --> what to employ and where to employ to

### some more links about what we can do
- https://blog.paralleldots.com/research/artificial-intelligence-can-make-public-transportation-safer/?source=post_page---------------------------